In [1]:
import pandas
import numpy as np
from numpy.linalg import inv
from sklearn import datasets, linear_model

In [2]:
data=pandas.read_csv('df_NaN.csv')

In [3]:
#set Developing to 1 and Developed to 0
data['Status'][data['Status'] == 'Developing']  = 0
data['Status'][data['Status'] == 'Developed']  = 1


#data normalization for further processing
#data.apply(normalization,axis=0)

D:\Users\ericl\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
D:\Users\ericl\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [4]:
#extract target data

target=data['Life expectancy ']
target=np.array(target)
#Ignore the country and year effect
data=data.drop(['Country','Year','Unnamed: 0','Life expectancy '],axis=1)
#normalize data


In [5]:
#extract normalization parameters and normalize original data
dataMax=np.array(data.max())
dataMin=np.array(data.min())
dataMean=np.array(data.mean())
data=data.apply(lambda x: (x - np.min(x)) / (np.max(x) - np.min(x)))
# is it necessary to normalize label data?
life=target.copy()
#target=(target-target.min())/(target.max()-target.min())

In [6]:
train_x=np.array(data)

In [7]:
def least_square(train_x, train_y):
    """
    use least square method to reach the regression model
    input：
          train_x, training data
          train_y, labels in training data
    """  
    weights = inv(np.dot(train_x.T ,train_x).astype(float)).dot(train_x.T).dot(train_y) 
    return weights.astype(float)

least_square_weights=least_square(train_x,target)
#prove to be very inaccurate

In [8]:
"""
def gradient_descent(train_x, train_y, maxCycle, alpha):
     
    numSamples, numFeatures = np.shape(train_x)
    weights = np.zeros((numFeatures,1))
     
    for i in range(maxCycle):
        h = train_x.dot(weights)
        err = h - train_y           
        weights = weights - (alpha*err.T.dot(train_x)).T
        print(err)
    return weights.astype(float)
"""

'\ndef gradient_descent(train_x, train_y, maxCycle, alpha):\n     \n    numSamples, numFeatures = np.shape(train_x)\n    weights = np.zeros((numFeatures,1))\n     \n    for i in range(maxCycle):\n        h = train_x.dot(weights)\n        err = h - train_y           \n        weights = weights - (alpha*err.T.dot(train_x)).T\n        print(err)\n    return weights.astype(float)\n'

In [25]:
#train a shallow neural network with pytorch


import torch
import sys
from torch.autograd import Variable

import torch .nn. functional as F

torch_x=Variable(torch.from_numpy(train_x.astype(float)),requires_grad = True)
torch_y=Variable(torch.from_numpy(target),requires_grad = False)

class Model(torch.nn.Module):
    def __init__(self):
        super(Model, self).__init__()
        self.fc1 = torch.nn.Linear(19,57).double()
        #self.fc2 = torch.nn.Linear(19,19).double()
        self.fc2 = torch.nn.Linear(57,1).double()
        
    def forward(self,x):
        x=F.relu(self.fc1(x))
        #x=F.relu(self.fc2(x))
        x=self.fc2(x)
        return x
    
model=Model()
criterion=torch.nn.MSELoss()
optimizer = torch.optim.SGD(model.parameters(), lr=0.01)
            
         
  

In [26]:
y_pred=model(torch_x)

In [27]:
y_pred

tensor([[ 0.0013],
        [-0.0079],
        [-0.0104],
        ...,
        [ 0.0648],
        [ 0.0123],
        [-0.0420]], dtype=torch.float64, grad_fn=<ThAddmmBackward>)

In [28]:
torch_y=torch_y.reshape([2938,1])

In [29]:
y_pred.shape

torch.Size([2938, 1])

In [30]:
T=200
B=100
NB=30
N=2938
for epoch in range(T):
    running_loss=0.0
    idxminibatches = np. random . permutation (NB)
    for k in range(NB):
        i = idxminibatches [k]
        idxsmp=np.arange(B*i,min(B*(i+1),N))
        inputs = torch_x[idxsmp]
        labels = torch_y[idxsmp]
        
        optimizer.zero_grad()

        y_pred=model(inputs)

        loss=criterion(y_pred,labels)
        
        loss.backward()

        optimizer.step()
        running_loss+=loss[0]
        if k==29:
            
            print(epoch,running_loss)
            running_loss=0.0



        


            

D:\Users\ericl\Anaconda3\lib\site-packages\ipykernel_launcher.py:23: UserWarning: invalid index of a 0-dim tensor. This will be an error in PyTorch 0.5. Use tensor.item() to convert a 0-dim tensor to a Python number


0 tensor(89180.9994, dtype=torch.float64, grad_fn=<ThAddBackward>)
1 tensor(37045.3149, dtype=torch.float64, grad_fn=<ThAddBackward>)
2 tensor(9327.3437, dtype=torch.float64, grad_fn=<ThAddBackward>)
3 tensor(4021.1235, dtype=torch.float64, grad_fn=<ThAddBackward>)
4 tensor(2594.2878, dtype=torch.float64, grad_fn=<ThAddBackward>)
5 tensor(2653.7137, dtype=torch.float64, grad_fn=<ThAddBackward>)
6 tensor(2245.8566, dtype=torch.float64, grad_fn=<ThAddBackward>)
7 tensor(1802.4337, dtype=torch.float64, grad_fn=<ThAddBackward>)
8 tensor(1877.6328, dtype=torch.float64, grad_fn=<ThAddBackward>)
9 tensor(657.9297, dtype=torch.float64, grad_fn=<ThAddBackward>)
10 tensor(648.8578, dtype=torch.float64, grad_fn=<ThAddBackward>)
11 tensor(1875.6460, dtype=torch.float64, grad_fn=<ThAddBackward>)
12 tensor(1284.7494, dtype=torch.float64, grad_fn=<ThAddBackward>)
13 tensor(1064.6004, dtype=torch.float64, grad_fn=<ThAddBackward>)
14 tensor(969.6589, dtype=torch.float64, grad_fn=<ThAddBackward>)
15 ten

134 tensor(363.8457, dtype=torch.float64, grad_fn=<ThAddBackward>)
135 tensor(301.5899, dtype=torch.float64, grad_fn=<ThAddBackward>)
136 tensor(350.5402, dtype=torch.float64, grad_fn=<ThAddBackward>)
137 tensor(366.8414, dtype=torch.float64, grad_fn=<ThAddBackward>)
138 tensor(388.9246, dtype=torch.float64, grad_fn=<ThAddBackward>)
139 tensor(338.3568, dtype=torch.float64, grad_fn=<ThAddBackward>)
140 tensor(325.8183, dtype=torch.float64, grad_fn=<ThAddBackward>)
141 tensor(357.3830, dtype=torch.float64, grad_fn=<ThAddBackward>)
142 tensor(308.2303, dtype=torch.float64, grad_fn=<ThAddBackward>)
143 tensor(381.6113, dtype=torch.float64, grad_fn=<ThAddBackward>)
144 tensor(384.7118, dtype=torch.float64, grad_fn=<ThAddBackward>)
145 tensor(350.2923, dtype=torch.float64, grad_fn=<ThAddBackward>)
146 tensor(314.0300, dtype=torch.float64, grad_fn=<ThAddBackward>)
147 tensor(302.5803, dtype=torch.float64, grad_fn=<ThAddBackward>)
148 tensor(351.1581, dtype=torch.float64, grad_fn=<ThAddBackwa

In [31]:
prediction=model(torch_x)

In [32]:
prediction

tensor([[62.9524],
        [63.6344],
        [63.9052],
        ...,
        [44.7711],
        [45.5484],
        [46.6790]], dtype=torch.float64, grad_fn=<ThAddmmBackward>)